In [17]:
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import requests
from selenium.webdriver.support import expected_conditions as EC

In [18]:
link="https://www.makaan.com/"
requests.get(link)

<Response [200]>

In [19]:
s = Service(r"C:\Users\bhupe\Downloads\chromedriver-win32\chromedriver-win32\chromedriver.exe")
driver = webdriver.Chrome(service=s)
driver.maximize_window()
driver.get(link)


In [20]:
search_bar_xpath = "/html/body/div[1]/main/div/section[1]/div[1]/div/div[3]/div[1]/div/div/div/div[2]/input"
search_bar = driver.find_element(By.XPATH, search_bar_xpath)
search_bar.send_keys("Bangalore")

result_row_class = "result-row"
# Wait for the "result-row" element to be clickable and then click it
result_row = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME, result_row_class)))
result_row.click()

search_button_xpath = "/html/body/div[1]/main/div/section[1]/div[1]/div/div[3]/div[3]/span[2]"
search_button = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, search_button_xpath)))
search_button.click()

# Wait for the "page numbers" element to be visible and then extract the number of pages
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

page_numbers_xpath = "/html/body/div[1]/main/div/div/div[2]/div/div[2]/div/div[3]/div[1]/div/ul/li[7]/a"
page_numbers_element = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, page_numbers_xpath)))
pages = int(page_numbers_element.text)
print(f'No of pages available--{pages}')

No of pages available--2163


In [21]:
dic={"property":[],"location":[],"price":[],"area":[],"furnished status":[],"additional details":[]}

In [ ]:
for page in range(1,pages):
    if page % 50 == 0:
        time.sleep(30)

    driver.get(f"https://www.makaan.com/bangalore-residential-property/buy-property-in-bangalore-city?page={page}&_=1725726991020")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    listings = driver.find_elements(By.CLASS_NAME, "cardholder")  # Assuming "cardholder" is a class that encapsulates each listing

    for listing in listings:
        try:
            location = listing.find_element(By.CLASS_NAME, "loclink").text
        except:
            location = "NULL"
        
        try:
            property_name = listing.find_element(By.CLASS_NAME, "title-line").text
        except:
            property_name = "NULL"
        
        try:
            price = listing.find_element(By.CLASS_NAME, "price").text
        except:
            price = "NULL"
        
        try:
            area = listing.find_element(By.CLASS_NAME, "size").text
        except:
            area = "NULL"
        
        try:
            furnished_status = listing.find_element(By.CLASS_NAME, "hcol.w44").text
        except:
            furnished_status = "NULL"
        
        try:
            additional_details = listing.find_element(By.CLASS_NAME, "listing-details").text
        except:
            additional_details = "NULL"

        # Append all values to the dictionary
        dic["location"].append(location)
        dic["property"].append(property_name)
        dic["price"].append(price)
        dic["area"].append(area)
        dic["furnished status"].append(furnished_status)
        dic["additional details"].append(additional_details)

    # Save to CSV when the data reaches 5000 entries
        if len(dic["location"]) == 5000:
            i=1
            file = pd.DataFrame(dic)
            print(f'-----------FILE ::: {i}--{5000*i} <-- Data Extracted')
            print(file.head())
            file.to_csv(f"makan{i}.csv", index=False)
            dic = {"property": [], "location": [], "price": [], "area": [], "furnished status": [], "additional details": []}
